In [ ]:
"""
To Make sure the Best Axon method works and also
to see if can decompose the axon any better 
(especially if can help with crosses and parts that aren't skeletonized             )

"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty/")

In [3]:
import datajoint_utils as du
minnie,schema = du.configure_minnie_vm()

INFO - 2021-02-25 07:41:21,639 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-02-25 07:41:21,641 - settings - Setting database.user to celiib
INFO - 2021-02-25 07:41:21,642 - settings - Setting database.password to newceliipass
INFO - 2021-02-25 07:41:21,648 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-02-25 07:41:21,649 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-25 07:41:21,664 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-02-25 07:41:22,071 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-25 07:41:22,111 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-25 07:41:22,514 - settings - Setting enable_python_native_blobs to True


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import networkx_utils as xu
import datajoint as dj
import neuron_visualizations as nviz
import neuron
import neuron_utils as nru
import proofreading_utils as pru
import classification_utils as clu
import system_utils as su
from importlib import reload
import trimesh_utils as tu

# Getting a more high fidelity axon skeleton

In [5]:
multi_axon_segs,multi_axon_splits = (minnie.AutoProofreadNeurons2() & "cell_type_predicted='excitatory'"
 & "n_axons>1").fetch("segment_id","split_index")

In [6]:
segment_id,split_index = 864691135426651890,0 #long axon good for proofreading
#segment_id, split_index = 864691135446640468,0 #comes off the side of dendrite

In [7]:
minnie.AutoProofreadNeurons2() & dict(segment_id = segment_id,
                                      split_index=split_index)

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,proof_version the version of code used for this cell typing classification,multiplicity the total number of neurons that came from the parent segment id,cell_type_predicted morphology predicted by classifier,spine_category,n_axons Number of axon candidates identified,n_apicals Number of apicals identified,axon_length length (in um) of the classified axon skeleton,axon_area # area (in um^2) of the classified axon,nucleus_id id of nucleus from the flat segmentation Equivalent to Allen: 'id'.,nuclei_distance the distance to the closest nuclei (even if no matching nuclei found),n_nuclei_in_radius the number of nuclei within the search radius of 15000 belonging to that segment,n_nuclei_in_bbox the number of nuclei within the bounding box of that soma,"soma_x x coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","soma_y y coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","soma_z z coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)",max_soma_n_faces The largest number of faces of the somas,max_soma_volume The largest volume of the somas the (volume in billions (10*9 nm^3)),n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,axon_angle_maximum the anlge of an identified axon,spine_density_classifier the number of spines divided by skeletal length for branches analyzed in classification,n_branches_processed the number branches used for the spine density analysis,skeletal_length_processed The total skeletal length of the viable branches used for the spine density analysis,n_branches_in_search_radius the number branches existing in the search radius used for spine density,skeletal_length_in_search_radius The total skeletal length of the branches existing in the search radius used for spine density,run_time the amount of time to run (seconds)
864691135426651890,0,0.25,30.00,0,0,0,1,1,excitatory,densely_spined,2,2,1135.6141158087469,1314.1385539441028,334179,1113.7,1,1,197203,187208,1517

In [8]:
du.plot_proofread_neuron(segment_id,
                        split_index=split_index,
                        plot_errored_faces=True,
                        plot_synapses=True,
                        plot_nuclei=True,
                        plot_axon=True)

Synapsse Results:
Total Number of Synapses = 129
n_valid_syn_centers_presyn = 28
n_errored_syn_centers_presyn = 15
n_valid_syn_centers_postsyn = 86
n_errored_syn_centers_postsyn = 0
Including axon mesh


/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [9]:
neuron_obj = du.decomposition_with_spine_recalculation(segment_id,
                                                      split_index=split_index)

Decompressing Neuron in minimal output mode...please wait
Pulled from Table Decomposition so setting split_index = 0
Number of Neurons found = 1


In [10]:
if type(neuron_obj) == tuple:
    neuron_obj = neuron_obj[0][split_index]

In [11]:
neuron_obj.segment_id

864691135426651890

In [12]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [13]:
clu.axon_classification(neuron_obj,
                        plot_candidates=True,
                       best_axon=True,
                       plot_axons=True)


Part 1: Axon like branchese 
{'L0': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37]), 'L2': array([0]), 'L5': array([1]), 'L7': array([0]), 'L8': array([0])}
Limb L0 soma angle: 164.26 
Limb L1 soma angle: 106.84 
Limb L2 soma angle: 17.13 
Limb L3 soma angle: 49.37 
Limb L4 soma angle: 136.14 
Limb L5 soma angle: 160.68 
Limb L6 soma angle: 85.6 
Limb L7 soma angle: 81.61 
Limb L8 soma angle: 95.44 

Part 2: possible_axon_limbs = [0, 1, 4, 5, 6, 7, 8]

Phase 2: Working on Limb L0
nodes_to_eliminate = []

Phase 2: Working on Limb L1
nodes_to_eliminate = [6]

Phase 2: Working on Limb L4
nodes_to_eliminate = [0]

Phase 2: Working on Limb L5
nodes_to_eliminate = [2]

Phase 2: Working on Limb L6
nodes_to_eliminate = []

Phase 2: Working on Limb L7
nodes_to_eliminate = [0]
The filtered network is empty so just leaving the candidates as empty lists

Phase 2: Working o

Final Candidates

Limb 0
Candidate 0 (forestgreen): array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37])

Limb 1
Candidate 0 (orange): array([4])
Candidate 1 (cyan): array([0, 1, 2, 3, 5])

Limb 4
Candidate 0 (gold): array([1])
Candidate 1 (brown): array([2, 3, 4])

Limb 5
Candidate 0 (cyan): array([0])
Candidate 1 (blue): array([1])

Limb 6
Candidate 0 (orange): array([0, 1, 2])

Limb 7
Candidate 0 (olive): array([0])

Limb 8
Candidate 0 (brown): array([0])
Part 3: axon_subgraph_candidates = {0: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]], 1: [[4], [0, 1, 2, 3, 5]], 4: [[1], [2, 3, 4]], 5: [[0], [1]], 6: [[0, 1, 2]], 7: [[0]], 8: [[0]]}


 --- Working on limb 0, candidate # 0
38 out of 38 branches are axons
Axon percentage = 1.0
curr_axon_angle_threshold = 95
candida

({'L0': array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
         13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
         26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37.])},
 {})

# Erasing the starting branch from the axon so can test if axon is not starter

In [15]:
axon_limb_branch_dict= neuron_obj.axon_limb_branch_dict

axon_limb_name = neuron_obj.axon_limb_name
curr_limb = neuron_obj[axon_limb_name]
axon_branches = axon_limb_branch_dict[axon_limb_name]
curr_limb.current_starting_node,neuron_obj.axon_starting_branch

neuron_obj[axon_limb_name][neuron_obj.axon_starting_branch].labels = []

In [16]:
nviz.plot_axon(neuron_obj)


 Working on visualization type: mesh


# The whole process of replacing the axon with higher fidelity

In [23]:
new_neuron_obj = pru.refine_axon_for_high_fidelity_skeleton(neuron_obj=neuron_obj,
    plot_new_axon_limb_correspondence = True,
    plot_new_limb_object = True,
    plot_final_revised_axon_branch=True,
     verbose = True,)



Starting axon branch was not the limb starting node so generating border vertices and computing upstream branch
upstream_node_to_axon_starting_branch = 17


  0%|          | 0/1 [00:00<?, ?it/s]

root_curr = [790860.5 811995.9 608767.9]
Time for preparing soma vertices and root: 0.0009968280792236328
invalidation_d = 2000
cc_vertex_thresh = 10


100%|██████████| 1/1 [00:01<00:00,  1.22s/it]

meshparty_segment_size = 100
Time for 1st pass MP skeletonization: 1.8477072715759277
connecting at the root
branches_touching_root = [96]
combining close nodes
combine_threshold = 1300
length of Graph = 13164
Working on path [546. 548. 550. 551.]
path_degrees = [3, 2, 2, 3]
Working on path [2840. 2847. 2855. 2864. 2882. 2901. 2904.]
path_degrees = [3, 2, 2, 2, 2, 2, 3]
Working on path [5264. 5282. 5298. 5301.]
path_degrees = [3, 2, 2, 3]
Working on path [5352. 5367. 5376.]
path_degrees = [3, 2, 3]
Working on path [8430. 8435. 8441. 8446. 8450. 8452.]
path_degrees = [3, 2, 2, 2, 2, 3]
Working on path [11939. 11950. 11956.]
path_degrees = [3, 2, 3]
Working on path [13169. 11967. 11973. 11981. 11986. 11992.]
path_degrees = [4, 2, 2, 2, 2, 3]
Working on path [13170. 11997. 12004. 12012. 12018. 12029. 12035.]
path_degrees = [5, 2, 2, 2, 2, 2, 3]
Working on path [12024. 12026. 12032. 13171.]
path_degrees = [3, 2, 2, 6]
Working on path [12213. 12218. 12224. 12230. 12234. 12239.]
path_degrees

...


AFTER face_lookup_resolved_test



Readjusting starting axon branch skeleton
Fixing the axon starting branch endpoint to align with upstream branch
Starting Branch 33 skeleton before fix: (1, 2, 3)
Starting Branch 33 skeleton AFTER fix: (2, 2, 3)


axon_limb_name= L0
Limb starting node = 17
Axon starting node = 15
axon_branches = [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37]
Using Original Starting Info
Leftover Nodes After Axon Deletion = [17]
New Node names = [81]
Leftover Nodes Mapping = {17: 81}
concept_network_dict = {0: [<networkx_utils.GraphOrderedEdges object at 0x7efdea827748>]}
New Limb Object


Working on limb L0 branch 33
Working on limb L0 branch 29
Working on limb L0 branch 34
Working on limb L0 branch 13
Working on limb L0 branch 28
Working on limb L0 branch 40
Working on limb L0 branch 50
Working on limb L0 branch 7
Working on limb L0 branch 12
Working on limb L0 branch 26
Working on limb L0 branch 30
Working on limb L0 branch 47
Working on limb L0 branch 51
Working on limb L0 branch 32
Working on limb L0 branch 35
Working on limb L0 branch 38
Working on limb L0 branch 48
Working on limb L0 branch 44
Working on limb L0 branch 52
Working on limb L0 branch 24
Working on limb L0 branch 31
Working on limb L0 branch 22
Working on limb L0 branch 37
Working on limb L0 branch 55
Working on limb L0 branch 56
Working on limb L0 branch 27
Working on limb L0 branch 41
Working on limb L0 branch 45
Working on limb L0 branch 54
Working on limb L0 branch 57
Working on limb L0 branch 10
Working on limb L0 branch 20
Working on limb L0 branch 25
Working on limb L0 branch 11
Working on limb


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [ ]:
pru.proofread_neuron(new_neuron_obj,
                    plot_limb_branch_filter_with_disconnect_effect=True)